# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from g_api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
# File to Load
file_to_load = "../WeatherPy/cities.csv"

# Read cities file and store into Pandas data frame
cities_df = pd.read_csv(file_to_load)
cities_df.head(10)


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Nanortalik,99,GL,1609795197,96,60.14,-45.24,27.12,8.41
1,1,Padang,90,ID,1609795199,88,-0.95,100.35,73.40,1.12
2,2,Carnarvon,75,AU,1609795200,94,-24.87,113.63,71.60,6.93
3,3,Vaini,75,TO,1609795203,88,-21.20,-175.20,80.60,9.17
4,4,Tehachapi,1,US,1609794962,44,35.13,-118.45,53.60,18.34
5,5,Husavik,20,IS,1609795205,64,66.04,-17.34,39.20,12.75
6,6,Puerto Ayora,20,EC,1609795207,65,-0.74,-90.35,80.60,8.05
7,7,East London,98,ZA,1609795209,72,-33.02,27.91,76.75,7.61
8,8,Nemuro,0,JP,1609795210,88,43.32,145.57,19.40,15.17
9,9,Río Gallegos,0,AR,1609795211,25,-51.62,-69.22,64.40,17.22


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [12]:
# Store 'Lat' and 'Lng' into  locations
locations = cities_df[["Lat", "Lng"]].astype(float)

#Store humidity
humidity = cities_df['Humidity'].astype(float)

# Create a Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [18]:
#Narrow down the DataFrame to find your ideal weather condition. For example:
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.
best_weather_cities_df = cities_df.loc[(cities_df["Max Temp"]>70) & (cities_df["Max Temp"]<90) & (cities_df['Wind Speed']<10)
                                      & (cities_df['Cloudiness'] == 0)]

best_weather_cities_df.count()

Unnamed: 0    24
City          24
Cloudiness    24
Country       23
Date          24
Humidity      24
Lat           24
Lng           24
Max Temp      24
Wind Speed    24
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [23]:
best_weather_cities_df['Hotel Name'] = ""

best_weather_cities_df.count()


<ipython-input-23-13bd5efed903>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_weather_cities_df['Hotel Name'] = ""


Unnamed: 0    24
City          24
Cloudiness    24
Country       23
Date          24
Humidity      24
Lat           24
Lng           24
Max Temp      24
Wind Speed    24
Hotel Name    24
dtype: int64

In [37]:
hotels = []

for i in range(len(best_weather_cities_df)):
    lat = best_weather_cities_df.iloc[i]['Lat']
    lng = best_weather_cities_df.iloc[i]['Lng']
    

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }

    # assemble url and make API request
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        hotels.append(response['results'][i]['name'])
        
    except:
        hotels.append("None")
  

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map

# Assign the marker layer to a variable
markers = gmaps.marker_layer(coordinates)
# Add the layer to the map
fig.add_layer(markers)
fig

# Display figure
